In [1]:
from astropy.io import fits
from astropy.wcs import WCS
import os
import glob
path_data = '/home/aellien/Euclid_LSB_DR1/data/EUC_EDF_N_bin4_10k2/'
path_analysis = '/home/aellien/Euclid_LSB_DR1/analysis'
path_cut = '/home/aellien/Euclid_LSB_DR1/data/EUC_EDF_N_bin4_2.5k2'

In [28]:
# Make tile region files

all_regions = [
        "# Region file format: DS9 version 4.1",
        "global color=green dashlist=8 3 width=1 font='helvetica 10 normal' select=1",
        "fk5" ]

for tile in glob.glob(path_data+'/*scaled.fits'):
    
    #print(tile)
    nf = tile.split('/')[-1]
    tile_id = nf.split('_')[4]
    hdu = fits.open(tile)
    data = hdu[1].data
    header = hdu[1].header
    wcs = WCS(header)
    ny, nx = data.shape  # image dimensions

    # Get world coordinates of image corners
    corners_pixel = [(0, 0), (nx - 1, 0), (nx - 1, ny - 1), (0, ny - 1)]
    corners_world = wcs.pixel_to_world_values([x for x, y in corners_pixel],
                                              [y for x, y in corners_pixel])
    # Build DS9 polygon region in world coordinates (assumes RA/Dec)
    ra = corners_world[0]
    dec = corners_world[1]
    region_lines = [
        "# Region file format: DS9 version 4.1",
        "global color=green dashlist=8 3 width=1 font='helvetica 10 normal' select=1",
        "fk5",
        "polygon(" + ",".join(f"{r},{d}" for r, d in zip(ra, dec)) + ") # text={%s}"%tile_id
    ]
    all_regions.append("polygon(" + ",".join(f"{r},{d}" for r, d in zip(ra, dec)) + ") # text={%s}"%tile_id)
    
    with open(os.path.join(path_data, 'EUC_VIS_LSB_ObsID_%s_scaled.reg'%tile_id), 'w') as f:
        f.write("\n".join(region_lines))

with open(os.path.join(path_data, 'all_tiles.reg'), 'w') as f:
        f.write("\n".join(all_regions))

print('done')

done


In [2]:
# Make cut region files

all_regions = [
        "# Region file format: DS9 version 4.1",
        "global color=yellow dashlist=8 3 width=1 font='helvetica 10 normal' select=1",
        "fk5" ]

for cut in glob.glob(path_cut+'/*scaled.cut*.fits'):
    
    nf = cut.split('/')[-1]
    tile_id = nf.split('_')[4]
    cut_id = nf.split('_')[5][10]
    hdu = fits.open(cut)
    data = hdu[0].data
    header = hdu[0].header
    wcs = WCS(header)
    ny, nx = data.shape  # image dimensions

    # Get world coordinates of image corners
    corners_pixel = [(0, 0), (nx - 1, 0), (nx - 1, ny - 1), (0, ny - 1)]
    corners_world = wcs.pixel_to_world_values([x for x, y in corners_pixel],
                                              [y for x, y in corners_pixel])
    # Build DS9 polygon region in world coordinates (assumes RA/Dec)
    ra = corners_world[0]
    dec = corners_world[1]
    region_lines = [
        "# Region file format: DS9 version 4.1",
        "global color=yellow dashlist=8 3 width=1 font='helvetica 10 normal' select=1",
        "fk5",
        "polygon(" + ",".join(f"{r},{d}" for r, d in zip(ra, dec)) + ") # text={%s}"%(tile_id+cut_id)
    ]
    all_regions.append("polygon(" + ",".join(f"{r},{d}" for r, d in zip(ra, dec)) + ") # text={%s}"%(tile_id+cut_id))
    
    with open(os.path.join(path_cut, 'EUC_VIS_LSB_ObsID_%s_scaled.cut%s.reg'%(tile_id, cut_id)), 'w') as f:
        f.write("\n".join(region_lines))

with open(os.path.join(path_cut, 'all_cut.reg'), 'w') as f:
        f.write("\n".join(all_regions))

print('done')

done
